<a href="https://colab.research.google.com/github/RobDBennett/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please ignore this part. I am following along with the code in the boxes below. This is largely for my notes :)

In [0]:
# concatenating is adding two things together. Simpliest is with the + operator.
# Can be tricky with dataframes, so we need to add two metrics. 
by_rows = pd.concat([df1, df2])
# .concat needs a list of dataframes, adds to rows by default- columns stay the same.
by_cols = pd.concat([df1, df2], axis=1)
# when concat frames that aren't the same structure, adds NaN to values.
# %cd us a directory commands, changing directory. !ls is listing contents.
# the % is a 'magic' command, not bash. Bash just runs script. Magic command, 
# changes the state of the underlying computer. Flags are added with a -letter



Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2020-05-06 19:31:01--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.8.86
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.8.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  89.7MB/s    in 2.2s    

2020-05-06 19:31:04 (89.7 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [0]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [0]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01


In [0]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

Generalized thoughts on the matter- 
Products are saved as an int in the products csv, and those ints are also called to in the orders-prior/train csv's. I'll need to merge the train/prior frames, and then I can search for how many times that item number showed up. I'll also need the products csv to find the appropriate numbers I am calling.
I will need the product_id column (and the product_name so it makes sense). I looked over the other columns and I don't think I'm not sure if I actually need both prior and train, as train is listed in the documentation as training data supplied to participants and the test was used for competitions. 

In [0]:
##### YOUR CODE HERE #####
import pandas as pd
import io
import numpy as np
products = pd.read_csv('products.csv')
print(products.shape)
products.head()


(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [0]:
order_products_prior = pd.read_csv('order_products__prior.csv')
print(order_products_prior.shape)
order_products_prior.head()

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [0]:
products = products.drop(columns=['aisle_id', 'department_id'])
products.head()

,product_id,product_name
0,1,Chocolate Sandwich Cookies
1,2,All-Seasons Salt
2,3,Robust Golden Unsweetened Oolong Tea
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...
4,5,Green Chile Anytime Sauce


In [0]:
order_products_prior = order_products_prior.drop(columns=['add_to_cart_order', 'reordered'])

In [0]:
prod = products
prior = pd.concat([order_products_prior, order_products_train])
together = pd.merge(prior, prod, on='product_id', how='inner')
together

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2,33120,NaN,NaN,Organic Egg Whites
1,26,33120,NaN,NaN,Organic Egg Whites
2,120,33120,NaN,NaN,Organic Egg Whites
3,327,33120,NaN,NaN,Organic Egg Whites
4,390,33120,NaN,NaN,Organic Egg Whites
...,...,...,...,...,...
33819101,2269288,27499,31.0,0.0,Non-Dairy Coconut Seven Layer Bar
33819102,2507248,27499,9.0,0.0,Non-Dairy Coconut Seven Layer Bar
33819103,2271346,37703,15.0,0.0,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...
33819104,3004244,43725,25.0,0.0,Sweetart Jelly Beans


In [0]:
together['product_name'].value_counts().head(10)

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code. Remember, if you haven't followed along typing out what we did during lecture, do that now to get more repetition with the syntax.
- Take table 2 (the transpose of table 1) and practice changing it into Tidy format and back again. You should not use the transpose operation anywhere in this code:
 - Table 2 --> Tidy
 - Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
%matplotlib inline
import pandas as pd
import numpy as np

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [0]:
# table2 -> tidy
tidy1 = table2.reset_index()
tidy1

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [0]:
tidy1 = tidy1.melt(id_vars= 'index', value_vars=['John Smith','Jane Doe','Mary Johnson'])
tidy1

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [0]:
tidy1 = tidy1[['variable','index','value']]
tidy1

,variable,index,value
0,John Smith,treatmenta,NaN
1,John Smith,treatmentb,2.0
2,Jane Doe,treatmenta,16.0
3,Jane Doe,treatmentb,11.0
4,Mary Johnson,treatmenta,3.0
5,Mary Johnson,treatmentb,1.0


In [0]:
tidy1 = tidy1.rename(columns={'variable':'name','index':'trt','value':'result'})
tidy1

,name,trt,result
0,John Smith,treatmenta,NaN
1,John Smith,treatmentb,2.0
2,Jane Doe,treatmenta,16.0
3,Jane Doe,treatmentb,11.0
4,Mary Johnson,treatmenta,3.0
5,Mary Johnson,treatmentb,1.0


In [0]:
tidy1.trt = tidy1.trt.str.replace('treatment','')
tidy1

,name,trt,result
0,John Smith,a,NaN
1,John Smith,b,2.0
2,Jane Doe,a,16.0
3,Jane Doe,b,11.0
4,Mary Johnson,a,3.0
5,Mary Johnson,b,1.0


In [0]:
# tidy -> table2
bady = tidy1.pivot_table(index='name',columns='trt',values='result')
bady

trt,a,b
name,,
Jane Doe,16.0,11.0
John Smith,NaN,2.0
Mary Johnson,3.0,1.0


In [0]:
import seaborn as sns; sns.set(style='ticks', color_codes=True)
flights = sns.load_dataset('flights')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# Flights Pivot Table
pilot = flights.pivot_table(index='year', columns='month', values='passengers')
pilot

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



Thoughts on this one-
So this is more or less the same process, but now I need to account for time of day ordered, which is in the orders csv. I think I can use my data from before, so that is going to be my first approach and just tack on a very truncated orders csv to my existing table.
First push through has basically the same list, just with dramatically less counts, which makes me wonder if the data I'm pulling is wrong. 


In [0]:
##### YOUR CODE HERE #####
short = orders.drop(columns=['user_id', 'eval_set','order_number','order_dow','days_since_prior_order'])
short

,order_id,order_hour_of_day
0,2539329,8
1,2398795,7
2,473747,12
3,2254736,7
4,431534,15
...,...,...
3421078,2266710,18
3421079,1854736,10
3421080,626363,12
3421081,2977660,12


In [0]:
final = pd.merge(together, short, on='order_id', how='inner')
final.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,order_hour_of_day
0,2,33120,NaN,NaN,Organic Egg Whites,9
1,2,28985,NaN,NaN,Michigan Organic Kale,9
2,2,9327,NaN,NaN,Garlic Powder,9
3,2,45918,NaN,NaN,Coconut Butter,9
4,2,30035,NaN,NaN,Natural Sweetener,9


In [0]:
final = final.drop(columns=['add_to_cart_order','reordered'])
final.head()

,order_id,product_id,product_name,order_hour_of_day
0,2,33120,Organic Egg Whites,9
1,2,28985,Michigan Organic Kale,9
2,2,9327,Garlic Powder,9
3,2,45918,Coconut Butter,9
4,2,30035,Natural Sweetener,9


In [0]:
test = final[(final['order_hour_of_day']) >= ]
test['product_name'].value_counts().head(30)

Banana                                  488376
Bag of Organic Bananas                  392093
Organic Strawberries                    273680
Organic Baby Spinach                    249867
Organic Hass Avocado                    219434
Organic Avocado                         183003
Large Lemon                             159765
Strawberries                            148475
Limes                                   145687
Organic Whole Milk                      141870
Organic Raspberries                     141648
Organic Yellow Onion                    116992
Organic Garlic                          113151
Organic Zucchini                        108647
Organic Blueberries                     104247
Cucumber Kirby                           98975
Organic Fuji Apple                       92310
Organic Lemon                            90588
Organic Grape Tomatoes                   87432
Apple Honeycrisp Organic                 86721
Seedless Red Grapes                      86140
Organic Cucum

In [0]:
hours = [21, 22]
testing = final[final.order_hour_of_day.isin(hours)]
testing['product_name'].value_counts().head(25)

Banana                      21397
Bag of Organic Bananas      17990
Organic Strawberries        13784
Organic Baby Spinach        11834
Organic Hass Avocado         9968
Organic Avocado              8166
Organic Whole Milk           7492
Organic Raspberries          7085
Large Lemon                  6921
Strawberries                 6886
Limes                        6441
Organic Zucchini             5507
Organic Blueberries          5488
Organic Yellow Onion         5285
Organic Garlic               5205
Cucumber Kirby               4646
Organic Grape Tomatoes       4444
Seedless Red Grapes          4331
Organic Baby Carrots         4077
Apple Honeycrisp Organic     4037
Carrots                      3992
Organic Cucumber             3988
Organic Lemon                3981
Organic Fuji Apple           3874
Honeycrisp Apple             3810
Name: product_name, dtype: int64

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####
